# Парсер новостных текстов с сайтов РБК 

## Импорт библиотек и описание классов

In [1]:
# Установка библиотек
!pip install bs4
!pip install openpyxl

In [4]:
# Импорт библиотек
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
#from IPython import display

Ниже описаны классы для извлечения данных. 

Используются поисковые движки сайтов, возвращающие по запросу json таблицы с информацией о статьях. 

При их использовании есть различные ограничения, например, на количество статей в запросе. Так на сайте РБК выдается максимум 20 статей.   

### Парсер для РБК



In [5]:
class rbc_parser:
    def __init__(self):
        pass
    
    
    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями
        """
        url = 'https://www.rbc.ru/search/ajax/?' +\
        'project={0}&'.format(param_dict['project']) +\
        'category={0}&'.format(param_dict['category']) +\
        'dateFrom={0}&'.format(param_dict['dateFrom']) +\
        'dateTo={0}&'.format(param_dict['dateTo']) +\
        'page={0}&'.format(param_dict['page']) +\
        'query={0}&'.format(param_dict['query']) +\
        'material={0}'.format(param_dict['material'])
        
        # 'offset={0}&'.format(param_dict['offset']) +\
        # 'limit={0}&'.format(param_dict['limit']) +\
        
        return url
    
    
    def _get_search_table(self, param_dict: dict,
                          include_text: bool = True) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей
        
        include_text: bool
        ### Если True, статьи возвращаются с текстами
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json()['items'])
        if include_text and not search_table.empty:
            get_text = lambda x: self._get_article_data(x['fronturl'])
            search_table[['overview', 'text']] = search_table.apply(get_text,
                                                                    axis=1).tolist()
        
        if 'publish_date_t' in search_table.columns:
            search_table.sort_values('publish_date_t', ignore_index=True)
            
        return search_table
    
    
    def _iterable_load_by_page(self, param_dict):
        param_copy = param_dict.copy()
        results = []
        
        result = self._get_search_table(param_copy)
        results.append(result)
        
        while not result.empty:
            param_copy['page'] = str(int(param_copy['page']) + 1)
            result = self._get_search_table(param_copy)
            results.append(result)
            
        results = pd.concat(results, axis=0, ignore_index=True)
        
        return results
    
    
    def _get_article_data(self, url: str):
        """
        Возвращает описание и текст статьи по ссылке
        """
        r = rq.get(url)
        soup = bs(r.text, features="lxml") # features="lxml" чтобы не было warning
        div_overview = soup.find('div', {'class': 'article__text__overview'})
        if div_overview:
            overview = div_overview.text.replace('<br />','\n').strip()
        else:
            overview = None
        p_text = soup.find_all('p')
        if p_text:
            text = ' '.join(map(lambda x:
                                x.text.replace('<br />','\n').strip(),
                                p_text))
        else:
            text = None
        
        return overview, text 
    
    def get_articles(self,
                     param_dict,
                     time_step = 1,
                     save_every = 5,
                     save_excel = True) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса 
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### query - поисковой запрос (ключевое слово), например, РБК
        ###### page - смещение поисковой выдачи (с шагом 20)
        
        ###### Deprecated:
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%d.%m.%Y')
        dateTo = datetime.strptime(param_copy['dateTo'], '%d.%m.%Y')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')
        
        out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime("%d.%m.%Y")
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime("%d.%m.%Y")
            print('Parsing articles from ' + param_copy['dateFrom'] +  ' to ' + param_copy['dateTo'])
            out = pd.concat([out, self._iterable_load_by_page(param_copy)], axis=0, ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime("%d.%m.%Y")
            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_excel("rbk_checkpoint_table.xlsx")
                print('Checkpoint saved!')
                save_counter = 0
        
        if save_excel:
            out.to_excel("rbc_{}_{}.xlsx".format(
                param_dict['dateFrom'],
                param_dict['dateTo']))
        print('Finish')
        
        return out

## Пример выгрузки данных

### РБК

* __project__ - проекты РБК. Возможные значения: ["rbcnews", "rbctv", "rbcstyle", "sport", "realty", "crypto", "autonews", "quote", "bc3", "trends"]
 
* __category__ - рубрики: ["TopRbcRu_economics", "TopRbcRu_auto", "TopRbcRu_business", "TopRbcRu_money", "TopRbcRu_realty", "TopRbcRu_society", "TopRbcRu_politics", "TopRbcRu_own_business", "TopRbcRu_specials", "TopRbcRu_technology_and_media", "TopRbcRu_finances"]

* __material__ - материалы: ["video", "quiz", "interview", "research", "card", "opinion", "multimedia", "short_news", "olympics_online", "online", "investigation", "rating", "article_specproject", "article", "story"]

* __dateFrom__ - с даты 

* __dateTo__ - по дату 

* __page__ - смещение запроса (с шагом 20)

* __Deprecated__: 

    * __offset__ - смещение поисковой выдачи

    * __limit__ - лимит запроса, максимум 20

_Чтобы не специфировать параметр, оставляем поле пустым_

In [8]:
# Задаем параметры запроса и складываем в param_dict
use_parser = "РБК"

query = 'РБК'
project = "crypto"
category = ""
material = ""
dateFrom = '2014-09-17'
dateTo = "2024-05-24"
page = 0

if use_parser == "РБК":
    param_dict = {'query'   : query, 
                  'project' : project,
                  'category': category,
                  'dateFrom': datetime.
                  strptime(dateFrom, '%Y-%m-%d').
                  strftime('%d.%m.%Y'),
                  'dateTo'  : datetime.
                  strptime(dateTo, '%Y-%m-%d').
                  strftime('%d.%m.%Y'),
                  'page'   : str(page),
                  'material': material}

print(use_parser, "- param_dict:", param_dict)

РБК - param_dict: {'query': 'РБК', 'project': 'crypto', 'category': '', 'dateFrom': '12.12.2020', 'dateTo': '24.05.2024', 'page': '0', 'material': ''}


In [7]:
# Пример того, как выглядит json таблица запроса по параметрам.
# Действует ограничение в 20 статей на 1 запрос (параметром limit)
assert use_parser == "РБК"
parser = rbc_parser()
tbl = parser._get_search_table(param_dict,
                               include_text = True) # Парсить текст статей
print(len(tbl))
tbl.head()

20


,id,project,project_nick,type,category,title,body,publish_date,publish_date_t,fronturl,picture,badge,pay_option,data,_score,overview,text
0,665031d49a794718b99ccaf1,Крипто,crypto,article,None,Курс биткоина упал до отметки в $67 тыс.,Первая криптовалюта теряет в цене на фоне одоб...,2024-05-24T10:15:03+03:00,1716534903,https://www.rbc.ru/crypto/news/665031d49a79471...,https://s0.rbk.ru/v6_top_pics/media/img/5/36/3...,None,free,None,1,Первая криптовалюта теряет в цене на фоне одоб...,"Получайте рассылку с новостями, которые касают..."
1,664fb5949a794711a972ca70,Крипто,crypto,article,None,В США одобрили запуск биржевых фондов для крип...,SEC США долго откладывала рассмотрение заявок ...,2024-05-24T00:44:54+03:00,1716500694,https://www.rbc.ru/crypto/news/664fb5949a79471...,https://s0.rbk.ru/v6_top_pics/media/img/6/08/3...,None,free,None,1,SEC США долго откладывала рассмотрение заявок ...,"Получайте рассылку с новостями, которые касают..."
2,664f5aec9a79473397e8516a,Крипто,crypto,article,None,Главные тренды рынка криптовалют в 2024 году. ...,Рассказываем о главных нарративах сферы крипто...,2024-05-23T19:21:44+03:00,1716481304,https://www.rbc.ru/crypto/news/664f5aec9a79473...,https://s0.rbk.ru/v6_top_pics/media/img/9/38/3...,None,free,None,1,Рассказываем о главных нарративах сферы крипто...,"Получайте рассылку с новостями, которые касают..."
3,664f31f39a79479518199960,Крипто,crypto,article,None,Эксперты назвали криптовалюту Solana подходяще...,Одобрение ETF на Ethereum в США может открыть ...,2024-05-23T17:07:43+03:00,1716473263,https://www.rbc.ru/crypto/news/664f31f39a79479...,https://s0.rbk.ru/v6_top_pics/media/img/4/41/3...,None,free,None,1,Одобрение ETF на Ethereum в США может открыть ...,"Получайте рассылку с новостями, которые касают..."
4,664f208f9a79473554e07838,Крипто,crypto,article,None,Что будет с курсом Ethereum после одобрения ЕT...,Эксперты прогнозируют стремительный рост курса...,2024-05-23T14:31:20+03:00,1716463880,https://www.rbc.ru/crypto/news/664f208f9a79473...,https://s0.rbk.ru/v6_top_pics/media/img/2/39/3...,None,free,None,1,Эксперты прогнозируют стремительный рост курса...,"Получайте рассылку с новостями, которые касают..."


In [10]:
%%time
# Пример работы программы итеративного сбора большого количества текстов статей
# Работает, конечно, очень долго :(
table = parser.get_articles(param_dict=param_dict,
                             time_step = 1, # Шаг - 7 дней, можно больше,
                                            # но есть риск отсечения статей в неделях, гдестатей больше 100
                             save_every = 5, # Сохранять чекпойнт каждые 5 шагов
                             save_excel = True) # Сохранить итоговый файл
print(len(table))
table.head()

Checkpoint saved!
Finish
7580
CPU times: total: 38min 14s
Wall time: 1h 27min 21s


,id,project,project_nick,type,category,title,body,publish_date,publish_date_t,fronturl,picture,badge,pay_option,data,_score,overview,text
0,5fbeaee59a794751e9dbeee3,Крипто,crypto,article,None,От тысячи пицц до квартиры. Что можно сейчас к...,10 лет назад первую криптовалюту можно было пр...,2020-12-13T11:30:00+03:00,1607848200,https://www.rbc.ru/crypto/news/5fbeaee59a79475...,https://s0.rbk.ru/v6_top_pics/media/img/0/30/7...,None,free,None,1,10 лет назад первую криптовалюту можно было пр...,"Получайте рассылку с новостями, которые касают..."
1,5fd3a1dc9a79475022221ca7,Крипто,crypto,article,None,Как выбрать перспективную криптовалюту. Советы...,На крипторынке есть почти 8 тыс. токенов. Поня...,2020-12-12T11:45:12+03:00,1607762712,https://www.rbc.ru/crypto/news/5fd3a1dc9a79475...,https://s0.rbk.ru/v6_top_pics/media/img/5/65/7...,None,free,None,1,На крипторынке есть почти 8 тыс. токенов. Поня...,"Получайте рассылку с новостями, которые касают..."
2,5fd8ac969a79472abf701992,Крипто,crypto,article,None,Появилась новая форма ICO. При чем тут суды и ...,"Проект Avalanche анонсировал запуск продукта, ...",2020-12-15T18:19:31+03:00,1608045571,https://www.rbc.ru/crypto/news/5fd8ac969a79472...,https://s0.rbk.ru/v6_top_pics/media/img/7/53/7...,None,free,None,1,"Проект Avalanche анонсировал запуск продукта, ...","Получайте рассылку с новостями, которые касают..."
3,5fd8be679a794732c30dd85c,Крипто,crypto,article,None,Депутаты предложили разрешить переводы физлица...,Новый законопроект направлен на устранение про...,2020-12-15T18:00:31+03:00,1608044431,https://www.rbc.ru/crypto/news/5fd8be679a79473...,https://s0.rbk.ru/v6_top_pics/media/img/9/70/7...,None,free,None,1,Новый законопроект направлен на устранение про...,"Получайте рассылку с новостями, которые касают..."
4,5fd8b07d9a79472c52ecdcdd,Крипто,crypto,article,None,"$4 млн за месяц, успех токена сооснователя App...","Манипулятор, который за 1 минуту завысил курс ...",2020-12-15T16:19:48+03:00,1608038388,https://www.rbc.ru/crypto/news/5fd8b07d9a79472...,https://s0.rbk.ru/v6_top_pics/media/img/0/68/7...,None,free,None,1,"Манипулятор, который за 1 минуту завысил курс ...","Получайте рассылку с новостями, которые касают..."
